### Import

In [1]:
import os
import re  
import fnmatch 
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

import scipy.stats as stats
from scipy.stats import chi2_contingency

from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD

from sklearn.ensemble import RandomForestClassifier

### Path

In [2]:
parquet_dir = '.../Data/parquet/'
output_file = '.../Data/'

### Read Population Labels

In [3]:
pop_labels = pd.read_csv(parquet_dir + 'igsr_samples.tsv', sep='\t')
pop_labels.head(3)

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
0,HG00271,male,SAME123417,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1,HG00276,female,SAME123424,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2,HG00288,female,SAME1839246,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."


### Read, Clean, and Merge SNPs

In [4]:
def read_and_clean_chromosome_files(folder_path):

    for chr_num in range(1, 23):
        
        file_path = f"{folder_path}/chr{chr_num}.parquet"
        df = pd.read_parquet(file_path, engine='pyarrow')
        
        missing_percentage = df.isnull().mean() * 100
        cols_to_keep = missing_percentage[missing_percentage <= 5].index
        cleaned_df = df[cols_to_keep]
        
        if chr_num == 1:
            final = cleaned_df.copy()
        else:
            final = final.merge(cleaned_df, on='Person_ID')
                
    return final

In [5]:
result_df = read_and_clean_chromosome_files(parquet_dir)
result_df = result_df.set_index('Person_ID')

In [6]:
result_df.head(3)

,rs3094315,rs3131972,rs12124819,rs11240777,rs6681049,rs4970383,rs4475691,rs7537756,rs13302982,rs1110052,...,rs10451,rs715586,rs8137951,rs2301584,rs756638,rs3810648,rs2285395,rs13056621,rs5771007,rs3888396
Person_ID,,,,,,,,,,,,,,,,,,,,,
HG00096,1,1,0,1,2,0,0,0,2,1,...,0,1,0,0,0,0,0,1,0,1
HG00097,2,2,0,0,2,1,1,1,2,0,...,0,0,0,0,0,0,0,1,0,0
HG00099,1,1,0,1,2,0,0,0,2,2,...,1,0,1,0,0,0,0,2,0,0


### Outlier Detection

Identify all values that are outside the range of 0 to 2 (inclusive), and set those values to NaN, which represents missing data in pandas DataFrames

In [7]:
def clean_snp_data(df):
    
    mask = (df < 0) | (df > 2)
    df[mask] = np.nan
    
    return df

result_df = clean_snp_data(result_df)

### Remove SNPs with Missing Values > 5% 

Remove: If a sample (row) or SNP (column) has a high percentage of missing values, you might consider removing it entirely from the dataset.

In [8]:
threshold = 0.05 * result_df.shape[0]  
result_df = result_df.dropna(axis='columns', thresh=threshold)

### Impute Missing Values with Mode

Impute: For SNPs with a small percentage of missing data, you can impute missing values. Common strategies include replacing missing values with the median or mode. In SNP data, using the most common genotype (0, 1, or 2) among the observed values for that SNP can be a sensible approach.

In [9]:
def selective_imputation(df):
    
    missing_columns = df.columns[df.isnull().any()].tolist()

    imputer = SimpleImputer(strategy='most_frequent')
    
    if missing_columns:
        df[missing_columns] = imputer.fit_transform(df[missing_columns])
    
    return df

result_df = selective_imputation(result_df)

### Minor Allele Frequency (MAF)

SNPs with a very low MAF are often excluded since they may not contribute much information for ancestry prediction and can lead to model overfitting. A common threshold is to exclude SNPs with a MAF less than 5%.

In [10]:
def filter_snps_by_maf_A(df, maf_threshold=0.05):

    maf_dict = {}
    total_alleles = 2 * len(df)
    
    for column in df.columns:
        
        counts = df[column].value_counts()
        
        freq_minor_allele = min((2 * counts.get(2, 0) + counts.get(1, 0)) / total_alleles,
                                (2 * counts.get(0, 0) + counts.get(1, 0)) / total_alleles)
        
        maf_dict[column] = freq_minor_allele
    
    columns_to_keep = [column for column, maf in maf_dict.items() if maf >= maf_threshold]
    
    filtered_df = df[columns_to_keep]
    
    return filtered_df

In [11]:
def filter_snps_by_maf_B(dataframe, maf_threshold=0.05):

    maf_dict = {}
    to_drop  = []

    for snp in dataframe.columns:

        genotype_counts = dataframe[snp].value_counts()
        
        n_AA = genotype_counts.get(0, 0)
        n_Aa = genotype_counts.get(1, 0)
        n_aa = genotype_counts.get(2, 0)
        
        total_alleles = 2 * (n_AA + n_Aa + n_aa)
        freq_A = (2 * n_AA + n_Aa) / total_alleles
        freq_a = 1 - freq_A  
        
        maf = min(freq_A, freq_a)
        maf_dict[snp] = maf
        
        if maf < maf_threshold:
            to_drop.append(snp)
    
    filtered_df = dataframe.drop(columns=to_drop)

    return filtered_df

In [12]:
result_df = filter_snps_by_maf_B(result_df, maf_threshold=0.05)
print(result_df.shape)

(2504, 823111)


### Hardy-Weinberg Equilibrium (HWE)

Deviations from HWE can indicate problems such as genotyping errors. While checking for HWE is more common in association studies, you might decide to perform this check and exclude SNPs not in equilibrium if you suspect data quality issues.

In [13]:
def filter_snps_by_hwe_A(df, hwe_p_threshold=0.01):

    columns_to_keep = []
    
    for column in df.columns:
        
        genotype_counts = df[column].value_counts().sort_index()

        for genotype in [0, 1, 2]:  
            if genotype not in genotype_counts.index:
                genotype_counts.at[genotype] = 0

        p = (2 * genotype_counts[0] + genotype_counts[1]) / (2 * sum(genotype_counts.values)) 
        q = 1 - p 

        expected_counts = pd.Series({0: (p ** 2)  * sum(genotype_counts.values),  
                                     1: 2 * p * q * sum(genotype_counts.values), 
                                     2: (q ** 2)  * sum(genotype_counts.values)})

        chi2, p_value = stats.chisquare(f_obs=genotype_counts, f_exp=expected_counts)

        if p_value >= hwe_p_threshold:
            columns_to_keep.append(column)
            
    filtered_df = df[columns_to_keep]
    
    return filtered_df

In [14]:
def filter_snps_by_hwe_B(dataframe, alpha=0.01):

    hwe_results = {}
    
    for snp in dataframe.columns:
        
        genotype_counts = dataframe[snp].value_counts()
        
        obs_AA = genotype_counts.get(0, 0)
        obs_Aa = genotype_counts.get(1, 0)
        obs_aa = genotype_counts.get(2, 0)
        
        total_alleles = 2 * (obs_AA + obs_Aa + obs_aa)
        p = (2 * obs_AA + obs_Aa) / total_alleles 
        q = 1 - p  
        
        exp_AA = (p ** 2)  * (obs_AA + obs_Aa + obs_aa)
        exp_Aa = 2 * p * q * (obs_AA + obs_Aa + obs_aa)
        exp_aa = (q ** 2)  * (obs_AA + obs_Aa + obs_aa)
        
        observed = np.array([obs_AA, obs_Aa, obs_aa])
        expected = np.array([exp_AA, exp_Aa, exp_aa])
        chi2, p_value = chi2_contingency([observed, expected], correction=False)[:2] 
        
        is_in_equilibrium = p_value >= alpha
        
        hwe_results[snp] = (chi2, p_value, is_in_equilibrium)
    
    return hwe_results

In [15]:
def filter_snps_from_hwe_B(dataframe, hwe_results):

    snps_to_keep = [snp for snp, (_, _, is_in_equilibrium) in hwe_results.items() if is_in_equilibrium]
    filtered_dataframe = dataframe[snps_to_keep]
    
    return filtered_dataframe

In [16]:
hwe_results = filter_snps_by_hwe_B(result_df, alpha=0.01)
result_df = filter_snps_from_hwe_B(result_df, hwe_results)
print(result_df.shape)

(2504, 499504)


### Variance Threshold Filtering 

Variance Threshold: Remove SNPs that have low variance across samples, as they might not be informative.

In [17]:
def variance_threshold_filtering(df, p):
    
    threshold = p * (1 - p)
    selector = VarianceThreshold(threshold)

    df_transformed = selector.fit_transform(df)
    df_transformed = pd.DataFrame(df_transformed, columns=df.columns[selector.get_support()], index= df.index)

    kept_columns = df.columns[selector.get_support()]
    dropped_columns = df.columns[~selector.get_support()]
    
    return df_transformed, kept_columns, dropped_columns

In [18]:
result_df, _, _ = variance_threshold_filtering(result_df, 0.85)
print(result_df.shape)

(2504, 477372)


In [19]:
result_df.head(3)

,rs12124819,rs6681049,rs4970383,rs4475691,rs7537756,rs3748597,rs28391282,rs2340592,rs1891910,rs3128117,...,rs739365,rs5770992,rs2040487,rs9628187,rs6010063,rs10451,rs715586,rs8137951,rs2301584,rs3810648
Person_ID,,,,,,,,,,,,,,,,,,,,,
HG00096,0.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,2.0,...,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
HG00097,0.0,2.0,1.0,1.0,1.0,2.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HG00099,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0


### Save Clean Data

In [20]:
transposed_df = result_df.transpose()
transposed_df.head(3)

Person_ID,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00105,HG00106,HG00107,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
rs12124819,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rs6681049,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
rs4970383,0.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0


In [21]:
table = pa.Table.from_pandas(transposed_df)
pq.write_table(table, parquet_dir + "Final_Data.parquet")